In [1]:
import os
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

#from skimage.filters.rank import entropy
import sample_rasters as sr
from rasterio.crs import CRS

from shapely.geometry import box

import planetary_computer as pc

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

itemids = pd.read_csv(os.path.join(os.getcwd(),'temp', 'aoi_naip_itemids.csv'))

csv_name = 'spectral_window_test_set.csv'
all_pts = pd.read_csv(os.path.join(os.getcwd(), csv_name))

# ***************************************************
# ***************************************************

In [3]:
t0 = time.time()
for i in [9]:
#for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids.itemid[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    # ---------------------------------------
    # find polygons for that NAIP scene
    poly_fp = sr.path_to_polygons(itemids.iloc[i].aoi_name, itemids.iloc[i].year)
    polys = gpd.read_file(poly_fp)
    
    scene_entrs = []
    # iterate through polygons in scene
    for j in [0]:
#    for j in range(len(polys)):
        # ---------------------------------------
        # create enlarged bounding box
        poly = polys.geometry[j]
        bbox = box(*poly.bounds)  
        # NEED TO MAKE bbox BIGGER
        reduce = gpd.GeoDataFrame({'geometry':[bbox]}, crs=polys.crs)
        reduce = reduce.to_crs(naip_rast_r.rio.crs)      

        # ---------------------------------------
        # clip NAIP scene, calculate entropy, and save
        rast = naip_rast_r.rio.clip_box(*reduce.total_bounds)
        rast_name = itemid + '_poly_'+str(j)
        sr.entropy_raster(raster = rast, rast_name=rast_name)
        fp = os.path.join(os.getcwd(),'temp', rast_name + '_entrs.tif')
    

        # ---------------------------------------
        # find points in that polygon
        pts_poly = all_pts.loc[ (all_pts['naip_id'] == itemid) & (all_pts['polygon_id'] == j)]
        crs = CRS.from_string(pts_poly.pts_crs.iloc[0])
        pts_poly_df = sr.geodataframe_from_csv(df = pts_poly, lon_label='x', lat_label='y', crs=crs)
        pts_col = pts_poly_df.to_crs(naip_rast_r.rio.crs).geometry

NameError: name 'geopandas' is not defined

In [8]:
import shapely
shapely.__version__

'1.8.4'

In [ ]:
        # ---------------------------------------
        # sample raster values from points
        rast_r = rasterio.open(fp)
        col_name = 'trial_entrs'
        scene_entrs.append(sr.sample_raster_from_pts(pts_col, rast_r, [col_name]))
        
        # ---------------------------------------
        # delete aux entropy raster
        os.remove(fp)
        
print(time.time() -t0)

In [ ]:
# TO DO :
# - figure out df concatenation with points in each polygon
# - add sampling for each band of the raster
# - add ndvi sampling

In [ ]:
all_pts.assign(trial = c)

In [ ]:
c = [8]*231368